In [112]:
from pyspark.sql.functions import col, size, split, when, concat_ws, substring, udf
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, FloatType
from pyspark.sql import SparkSession
import pandas as pd
import time

In [113]:
json_path = 'C:/Files/College/GP/UCSDBooks/Downloads/goodreads_reviews_dedup.json'
csv_path = '../preprocessed_original/reviewsFull.csv'
bookidsPath = '../preprocessed_original/book_ids.csv'
booksPath = "C:/Files/College/GP/UCSDBooks/Books/goodreads_books.json"
booksSentPath = "C:/Files/College/GP/Repo/BookRecommendationSystem/Recommender/preprocessed_original/book_sentiment.csv"
genresPath = "C:/Files/College/GP/UCSDBooks/Books/goodreads_book_genres_initial.json"
columns = ['book_id', 'user_id', 'rating']
json_data = []

In [114]:
spark = SparkSession.builder.appName("Read JSON").getOrCreate()
spark

In [115]:
dfBooksSent = spark.read.csv(booksSentPath, header=True, inferSchema=True)
# rename book_id to book_id2
dfBooksSent = dfBooksSent.withColumnRenamed("book_id", "book_id2")
dfBooksSent.head(15)

[Row(book_id2='10000761', sentiment=0),
 Row(book_id2='10000761', sentiment=1),
 Row(book_id2='10000761', sentiment=0),
 Row(book_id2='10000761', sentiment=0),
 Row(book_id2='1002783', sentiment=1),
 Row(book_id2='1002783', sentiment=1),
 Row(book_id2='1002783', sentiment=1),
 Row(book_id2='10029674', sentiment=0),
 Row(book_id2='10029674', sentiment=1),
 Row(book_id2='1003397', sentiment=1),
 Row(book_id2='10046808', sentiment=1),
 Row(book_id2='10051877', sentiment=1),
 Row(book_id2='100553', sentiment=1),
 Row(book_id2='100553', sentiment=1),
 Row(book_id2='100553', sentiment=1)]

In [116]:
# combine all rows with the same book_id and get their count and total sentiment
dfBooksSent = dfBooksSent.groupBy("book_id2").agg({"sentiment": "sum", "book_id2": "count"})

In [117]:
print(dfBooksSent.count())
dfBooksSent.head(5)

1251905


[Row(book_id2='10000761', count(book_id2)=4, sum(sentiment)=1),
 Row(book_id2='1002783', count(book_id2)=3, sum(sentiment)=3),
 Row(book_id2='10029674', count(book_id2)=2, sum(sentiment)=1),
 Row(book_id2='1003397', count(book_id2)=1, sum(sentiment)=1),
 Row(book_id2='10046808', count(book_id2)=1, sum(sentiment)=1)]

In [118]:
# rename count(book_id2) to rating_count and sum(sentiment) to rating_sum
dfBooksSent = dfBooksSent.withColumnRenamed("count(book_id2)", "sentiment_count")
dfBooksSent = dfBooksSent.withColumnRenamed("sum(sentiment)", "sentiment_sum")
# calculate the average sentiment
dfBooksSent = dfBooksSent.withColumn("sentiment_avg", dfBooksSent.sentiment_sum / dfBooksSent.sentiment_count)
dfBooksSent.head(5)

[Row(book_id2='10000761', sentiment_count=4, sentiment_sum=1, sentiment_avg=0.25),
 Row(book_id2='1002783', sentiment_count=3, sentiment_sum=3, sentiment_avg=1.0),
 Row(book_id2='10029674', sentiment_count=2, sentiment_sum=1, sentiment_avg=0.5),
 Row(book_id2='1003397', sentiment_count=1, sentiment_sum=1, sentiment_avg=1.0),
 Row(book_id2='10046808', sentiment_count=1, sentiment_sum=1, sentiment_avg=1.0)]

In [119]:
# check if there are any ratings avg that are null
dfBooksSent.filter(dfBooksSent.sentiment_avg.isNull()).count()


0

In [120]:
# check if there are any ratings avg that are greater than 1
dfBooksSent.filter(dfBooksSent.sentiment_avg > 1).count()

0

In [121]:
dfBooks = spark.read.json(booksPath)
#dfBooks.head(5)



[Row(asin='', authors=[Row(author_id='604031', role='')], average_rating='4.00', book_id='5333265', country_code='US', description='', edition_information='', format='Paperback', image_url='https://images.gr-assets.com/books/1310220028m/5333265.jpg', is_ebook='false', isbn='0312853122', isbn13='9780312853129', kindle_asin='', language_code='', link='https://www.goodreads.com/book/show/5333265-w-c-fields', num_pages='256', popular_shelves=[Row(count='3', name='to-read'), Row(count='1', name='p'), Row(count='1', name='collection'), Row(count='1', name='w-c-fields'), Row(count='1', name='biography')], publication_day='1', publication_month='9', publication_year='1984', publisher="St. Martin's Press", ratings_count='3', series=[], similar_books=[], text_reviews_count='1', title='W.C. Fields: A Life on Film', title_without_series='W.C. Fields: A Life on Film', url='https://www.goodreads.com/book/show/5333265-w-c-fields', work_id='5400751'),
 Row(asin='', authors=[Row(author_id='626222', rol

In [122]:
# [Row(asin='', authors=[Row(author_id='604031', role='')], average_rating='4.00', book_id='5333265', country_code='US', description='', edition_information='', format='Paperback', image_url='https://images.gr-assets.com/books/1310220028m/5333265.jpg', is_ebook='false', isbn='0312853122', isbn13='9780312853129', kindle_asin='', language_code='', link='https://www.goodreads.com/book/show/5333265-w-c-fields', num_pages='256', popular_shelves=[Row(count='3', name='to-read'), Row(count='1', name='p'), Row(count='1', name='collection'), Row(count='1', name='w-c-fields'), Row(count='1', name='biography')], publication_day='1', publication_month='9', publication_year='1984', publisher="St. Martin's Press", ratings_count='3', series=[], similar_books=[], text_reviews_count='1', title='W.C. Fields: A Life on Film', title_without_series='W.C. Fields: A Life on Film', url='https://www.goodreads.com/book/show/5333265-w-c-fields', work_id='5400751'),
#  Row(asin='', authors=[Row(author_id='626222', role='')], average_rating='3.23', book_id='1333909', country_code='US', description='Anita Diamant\'s international bestseller "The Red Tent" brilliantly re-created the ancient world of womanhood. Diamant brings her remarkable storytelling skills to "Good Harbor" -- offering insight to the precarious balance of marriage and career, motherhood and friendship in the world of modern women. The seaside town of Gloucester, Massachusetts is a place where the smell of the ocean lingers in the air and the rocky coast glistens in the Atlantic sunshine. When longtime Gloucester-resident Kathleen Levine is diagnosed with breast cancer, her life is thrown into turmoil. Frightened and burdened by secrets, she meets Joyce Tabachnik -- a freelance writer with literary aspirations -- and a once-in-a-lifetime friendship is born. Joyce has just bought a small house in Gloucester, where she hopes to write as well as vacation with her family. Like Kathleen, Joyce is at a fragile place in her life.\nA mutual love for books, humor, and the beauty of the natural world brings the two women together. They share their personal histories, and help each other to confront scars left by old emotional wounds.\nWith her own trademark wisdom and humor, Diamant considers the nature, strength, and necessity of adult female friendship. "Good Harbor" examines the tragedy of loss, the insidious nature of family secrets, as well as the redemptive power of friendship.', edition_information='Abridged', format='Audio CD', image_url='https://s.gr-assets.com/assets/nophoto/book/111x148-bcc042a9c91a29c1d680899eff700a03.png', is_ebook='false', isbn='0743509986', isbn13='9780743509985', kindle_asin='B000FC0PBC', language_code='', link='https://www.goodreads.com/book/show/1333909.Good_Harbor', num_pages='', popular_shelves=[Row(count='2634', name='to-read'), Row(count='160', name='fiction'), Row(count='92', name='currently-reading'), Row(count='25', name='contemporary-fiction'), Row(count='22', name='owned'), Row(count='19', name='book-club'), Row(count='17', name='chick-lit'), Row(count='16', name='friendship'), Row(count='15', name='books-i-own'), Row(count='12', name='novel'), Row(count='12', name='general-fiction'), Row(count='11', name='anita-diamant'), Row(count='10', name='owned-books'), Row(count='10', name='contemporary'), Row(count='10', name='women'), Row(count='8', name='adult'), Row(count='7', name='default'), Row(count='7', name='favorites'), Row(count='7', name='didn-t-finish'), Row(count='6', name='own-it'), Row(count='6', name='library'), Row(count='6', name='new-england'), Row(count='6', name='bookclub'), Row(count='6', name='women-s-fiction'), Row(count='5', name='breast-cancer'), Row(count='5', name='beach-ocean'), Row(count='5', name='audio-book'), Row(count='5', name='historical-fiction'), Row(count='5', name='my-library'), Row(count='5', name='2006'), Row(count='5', name='literature'), Row(count='5', name='audiobook'), Row(count='4', name='read-2015'), Row(count='4', name='diamant'), Row(count='4', name='audio-books'), Row(count='4', name='diamant-anita'), Row(count='4', name='adult-fiction'), Row(count='4', name='my-books'), Row(count='4', name='jewish'), Row(count='4', name='novels'), Row(count='4', name='library-book'), Row(count='4', name='2002'), Row(count='3', name='read-2016'), Row(count='3', name='relationships'), Row(count='3', name='female-friendship'), Row(count='3', name='favorite-authors'), Row(count='3', name='kindle'), Row(count='3', name='have'), Row(count='3', name='strong-women'), Row(count='3', name='cancer'), Row(count='3', name='own-to-read'), Row(count='3', name='audiobooks'), Row(count='3', name='read-2011'), Row(count='3', name='audio'), Row(count='3', name='drama'), Row(count='3', name='read-2009'), Row(count='3', name='i-own'), Row(count='3', name='book-group'), Row(count='3', name='american'), Row(count='3', name='fiction-general'), Row(count='3', name='chicklit'), Row(count='3', name='book-club-books'), Row(count='2', name='hardback'), Row(count='2', name='my-personal-library'), Row(count='2', name='read-in-2017'), Row(count='2', name='read-2017'), Row(count='2', name='the-lewis-library'), Row(count='2', name='ficton'), Row(count='2', name='home'), Row(count='2', name='books'), Row(count='2', name='read-in-2015'), Row(count='2', name='listened'), Row(count='2', name='ebook'), Row(count='2', name='bookclub-books'), Row(count='2', name='to-buy'), Row(count='2', name='my-home-library'), Row(count='2', name='summer-reading'), Row(count='2', name='home-bookshelf'), Row(count='2', name='family'), Row(count='2', name='tbi-book-club'), Row(count='2', name='wish-list'), Row(count='2', name='own-book'), Row(count='2', name='unfinished'), Row(count='2', name='fiction-contemporary'), Row(count='2', name='adult-books'), Row(count='2', name='books-i-have'), Row(count='2', name='paperback'), Row(count='2', name='books-in-my-library'), Row(count='2', name='summer-reads'), Row(count='2', name='read-in-2011'), Row(count='2', name='read-in-2010'), Row(count='2', name='gave-up'), Row(count='2', name='book-club-picks'), Row(count='2', name='spiritual'), Row(count='2', name='realistic-fiction'), Row(count='2', name='beach-read'), Row(count='2', name='unread-books-i-own'), Row(count='2', name='read-in-2008'), Row(count='2', name='in-my-library'), Row(count='2', name='general')], publication_day='1', publication_month='10', publication_year='2001', publisher='Simon & Schuster Audio', ratings_count='10', series=[], similar_books=['8709549', '17074050', '28937', '158816', '228563', '11296581', '1073987', '7298465', '1274862', '18290554', '86382', '343067', '2774907', '663892', '1233981', '298912', '401091', '307575'], text_reviews_count='6', title='Good Harbor', title_without_series='Good Harbor', url='https://www.goodreads.com/book/show/1333909.Good_Harbor', work_id='1323437'),
#  Row(asin='B00071IKUY', authors=[Row(author_id='10333', role='')], average_rating='4.03', book_id='7327624', country_code='US', description='Omnibus book club edition containing the Ladies of Madrigyn and the Witches of Wenshar.', edition_information='Book Club Edition', format='Hardcover', image_url='https://images.gr-assets.com/books/1304100136m/7327624.jpg', is_ebook='false', isbn='', isbn13='', kindle_asin='', language_code='eng', link='https://www.goodreads.com/book/show/7327624-the-unschooled-wizard', num_pages='600', popular_shelves=[Row(count='58', name='to-read'), Row(count='15', name='fantasy'), Row(count='6', name='fiction'), Row(count='5', name='owned'), Row(count='3', name='hardcover'), Row(count='2', name='shelfari-favorites'), Row(count='2', name='series'), Row(count='1', name='might-read'), Row(count='1', name='dnf-d'), Row(count='1', name='hambly-barbara'), Row(count='1', name='strong-females'), Row(count='1', name='first-in-series'), Row(count='1', name='fantasy-sword-sorcery'), Row(count='1', name='no-thanks-series-collections-boxes'), Row(count='1', name='fantasy-all'), Row(count='1', name='entertaining-but-limited'), Row(count='1', name='kate-own'), Row(count='1', name='fantasy-epic'), Row(count='1', name='e-book'), Row(count='1', name='compliation'), Row(count='1', name='my-books'), Row(count='1', name='books-i-own-but-have-not-read'), Row(count='1', name='everything-owned'), Row(count='1', name='books-to-find'), Row(count='1', name='i-own-it'), Row(count='1', name='favorite'), Row(count='1', name='not-read'), Row(count='1', name='read-some-day'), Row(count='1', name='library'), Row(count='1', name='audiobooks'), Row(count='1', name='status-borrowed'), Row(count='1', name='paranormal-mystery'), Row(count='1', name='owned-books'), Row(count='1', name='spec-fic-fantasy-epic'), Row(count='1', name='spec-fic-fantasy'), Row(count='1', name='spec-fic-awd-locus-nom'), Row(count='1', name='spec-fic'), Row(count='1', name='01'), Row(count='1', name='hardbacks'), Row(count='1', name='paper'), Row(count='1', name='german'), Row(count='1', name='hardback'), Row(count='1', name='physical-scifi-fantasy'), Row(count='1', name='childhood-favorites'), Row(count='1', name='bundle-same-author'), Row(count='1', name='aa-sifi-fantasy'), Row(count='1', name='ready-to-read'), Row(count='1', name='bought-on-flee-markets'), Row(count='1', name='fantasy-general'), Row(count='1', name='hardcopy'), Row(count='1', name='box-2'), Row(count='1', name='unfinished'), Row(count='1', name='magic'), Row(count='1', name='duplicates'), Row(count='1', name='favorites'), Row(count='1', name='books-i-own'), Row(count='1', name='fantasy-classic'), Row(count='1', name='own-hard-copy'), Row(count='1', name='fantasy-read'), Row(count='1', name='book-club-edition'), Row(count='1', name='sci-fi-or-fantasy'), Row(count='1', name='fiction-fantasy'), Row(count='1', name='fiction-literature-poetry'), Row(count='1', name='paused-hiatus'), Row(count='1', name='status—borrowed'), Row(count='1', name='recs-fantasy'), Row(count='1', name='fantasy-scifi'), Row(count='1', name='omnibus'), Row(count='1', name='speculative'), Row(count='1', name='sf--fantasy'), Row(count='1', name='in-my-home-library'), Row(count='1', name='fant-myth-para-vamps'), Row(count='1', name='read-in-my-20s')], publication_day='', publication_month='', publication_year='1987', publisher='Nelson Doubleday, Inc.', ratings_count='140', series=['189911'], similar_books=['19997', '828466', '1569323', '425389', '1176674', '262740', '3743837', '880461', '2292726', '1883810', '1808197', '625150', '1988046', '390170', '2620131', '383106', '1597281'], text_reviews_count='7', title='The Unschooled Wizard (Sun Wolf and Starhawk, #1-2)', title_without_series='The Unschooled Wizard (Sun Wolf and Starhawk, #1-2)', url='https://www.goodreads.com/book/show/7327624-the-unschooled-wizard', work_id='8948723'),
#  Row(asin='', authors=[Row(author_id='9212', role='')], average_rating='3.49', book_id='6066819', country_code='US', description="Addie Downs and Valerie Adler were eight when they first met and decided to be best friends forever. But, in the wake of tragedy and betrayal during their teenage years, everything changed. Val went on to fame and fortune. Addie stayed behind in their small Midwestern town. Destiny, however, had more in store for these two. And when, twenty-five years later, Val shows up at Addie's front door with blood on her coat and terror on her face, it is the beginning of a wild adventure for two women joined by love and history who find strength together that they could not find alone.", edition_information='', format='Hardcover', image_url='https://s.gr-assets.com/assets/nophoto/book/111x148-bcc042a9c91a29c1d680899eff700a03.png', is_ebook='false', isbn='0743294297', isbn13='9780743294294', kindle_asin='B002ENBLOK', language_code='eng', link='https://www.goodreads.com/book/show/6066819-best-friends-forever', num_pages='368', popular_shelves=[Row(count='7615', name='to-read'), Row(count='728', name='chick-lit'), Row(count='673', name='currently-reading'), Row(count='404', name='fiction'), Row(count='152', name='books-i-own'), Row(count='119', name='jennifer-weiner'), Row(count='82', name='chicklit'), Row(count='76', name='owned'), Row(count='72', name='book-club'), Row(count='72', name='friendship'), Row(count='55', name='adult-fiction'), Row(count='54', name='chic-lit'), Row(count='51', name='favorites'), Row(count='50', name='adult'), Row(count='43', name='contemporary'), Row(count='39', name='read-in-2010'), Row(count='37', name='library'), Row(count='35', name='read-in-2009'), Row(count='34', name='owned-books'), Row(count='33', name='default'), Row(count='29', name='women-s-fiction'), Row(count='27', name='audiobooks'), Row(count='26', name='contemporary-fiction'), Row(count='26', name='audio'), Row(count='26', name='beach-reads'), Row(count='25', name='my-books'), Row(count='25', name='audiobook'), Row(count='23', name='romance'), Row(count='23', name='general-fiction'), Row(count='21', name='i-own'), Row(count='20', name='audio-book'), Row(count='20', name='humor'), Row(count='18', name='read-in-2011'), Row(count='18', name='ebooks'), Row(count='18', name='womens-fiction'), Row(count='18', name='kindle'), Row(count='18', name='novels'), Row(count='17', name='have'), Row(count='16', name='did-not-finish'), Row(count='16', name='read-2010'), Row(count='16', name='drama'), Row(count='15', name='read-in-2012'), Row(count='15', name='audio-books'), Row(count='15', name='guilty-pleasures'), Row(count='15', name='book-club-books'), Row(count='15', name='own-it'), Row(count='14', name='to-buy'), Row(count='13', name='read-2009'), Row(count='12', name='books-i-have'), Row(count='12', name='favorite-authors'), Row(count='12', name='weiner'), Row(count='11', name='read-in-2013'), Row(count='11', name='own-to-read'), Row(count='11', name='2010-books'), Row(count='11', name='library-book'), Row(count='11', name='women'), Row(count='10', name='relationships'), Row(count='10', name='mystery'), Row(count='10', name='fluff'), Row(count='10', name='chick'), Row(count='10', name='beach-read'), Row(count='10', name='borrowed'), Row(count='10', name='my-library'), Row(count='10', name='2009-reads'), Row(count='10', name='wish-list'), Row(count='10', name='want-to-read'), Row(count='9', name='finished'), Row(count='9', name='on-my-shelf'), Row(count='9', name='didn-t-finish'), Row(count='9', name='ebook'), Row(count='9', name='2009-books'), Row(count='8', name='dysfunctional-families'), Row(count='8', name='read-in-2015'), Row(count='8', name='read-2011'), Row(count='8', name='family'), Row(count='8', name='abandoned'), Row(count='8', name='light-reading'), Row(count='8', name='e-books'), Row(count='8', name='books-read-in-2010'), Row(count='8', name='books-read-in-2009'), Row(count='8', name='adult-books'), Row(count='7', name='read-in-2016'), Row(count='7', name='read-2015'), Row(count='7', name='2014-reads'), Row(count='7', name='weiner-jennifer'), Row(count='7', name='coming-of-age'), Row(count='7', name='audible'), Row(count='7', name='paperback'), Row(count='7', name='books'), Row(count='7', name='on-the-shelf'), Row(count='7', name='purchased'), Row(count='7', name='2010-read'), Row(count='7', name='lit'), Row(count='7', name='already-own'), Row(count='7', name='friends'), Row(count='6', name='shelfari-favorites'), Row(count='6', name='bullying'), Row(count='6', name='tbr'), Row(count='6', name='on-my-bookshelf'), Row(count='6', name='bookworm-bitches')], publication_day='14', publication_month='7', publication_year='2009', publisher='Atria Books', ratings_count='51184', series=[], similar_books=['6604176', '6054190', '2285777', '82641', '7569453', '7010683', '5941079', '7052976', '227709', '6389685', '5456247', '3134684'], text_reviews_count='3282', title='Best Friends Forever', title_without_series='Best Friends Forever', url='https://www.goodreads.com/book/show/6066819-best-friends-forever', work_id='6243154'),
#  Row(asin='', authors=[Row(author_id='149918', role='')], average_rating='3.40', book_id='287140', country_code='US', description='', edition_information='', format='', image_url='https://images.gr-assets.com/books/1413219371m/287140.jpg', is_ebook='false', isbn='0850308712', isbn13='9780850308716', kindle_asin='', language_code='', link='https://www.goodreads.com/book/show/287140.Runic_Astrology', num_pages='', popular_shelves=[Row(count='32', name='to-read'), Row(count='3', name='runes'), Row(count='2', name='owned'), Row(count='2', name='nonfiction'), Row(count='1', name='kill-it-with-fire'), Row(count='1', name='magic-grimoires-witchcraft'), Row(count='1', name='german-and-northern-culture'), Row(count='1', name='astrology'), Row(count='1', name='occult-magick'), Row(count='1', name='non-fiction'), Row(count='1', name='spirit'), Row(count='1', name='divination'), Row(count='1', name='asatru'), Row(count='1', name='heathen-norse-paganism-read'), Row(count='1', name='to-find'), Row(count='1', name='awaiting'), Row(count='1', name='witchery-and-paganism'), Row(count='1', name='default'), Row(count='1', name='it-wikipedia'), Row(count='1', name='owned-books'), Row(count='1', name='on-my-shelf-home-library'), Row(count='1', name='magick-and-myth')], publication_day='', publication_month='', publication_year='', publisher='', ratings_count='15', series=[], similar_books=[], text_reviews_count='5', title='Runic Astrology: Starcraft and Timekeeping in the Northern Tradition', title_without_series='Runic Astrology: Starcraft and Timekeeping in the Northern Tradition', url='https://www.goodreads.com/book/show/287140.Runic_Astrology', work_id='278577')]

# make authors just a list of author_ids
dfBooks = dfBooks.withColumn("authors", dfBooks.authors.author_id)
#dfBooks.head(5)


In [123]:
# only keep authors, book_id, descritption, image_url, isbn, isbn13, num_pages, publication_year, ratings_count
dfBooks = dfBooks.select("authors", "book_id", "description", "image_url", "isbn", "isbn13", "num_pages", "publication_year", "title", "average_rating", "ratings_count")
dfBooks.head(5)

[Row(authors=['604031'], book_id='5333265', description='', image_url='https://images.gr-assets.com/books/1310220028m/5333265.jpg', isbn='0312853122', isbn13='9780312853129', num_pages='256', publication_year='1984', title='W.C. Fields: A Life on Film', average_rating='4.00', ratings_count='3'),
 Row(authors=['626222'], book_id='1333909', description='Anita Diamant\'s international bestseller "The Red Tent" brilliantly re-created the ancient world of womanhood. Diamant brings her remarkable storytelling skills to "Good Harbor" -- offering insight to the precarious balance of marriage and career, motherhood and friendship in the world of modern women. The seaside town of Gloucester, Massachusetts is a place where the smell of the ocean lingers in the air and the rocky coast glistens in the Atlantic sunshine. When longtime Gloucester-resident Kathleen Levine is diagnosed with breast cancer, her life is thrown into turmoil. Frightened and burdened by secrets, she meets Joyce Tabachnik -- 

In [124]:
# merge books and booksSent
dfBooks = dfBooks.join(dfBooksSent, dfBooks.book_id == dfBooksSent.book_id2, how='left')
dfBooks.head(5)

[Row(authors=['415967'], book_id='10695227', description='Trying to work things out with Nash--her maybe boyfriend--is hard enough for Kaylee Cavanaugh. She can\'t just pretend nothing happened. But "complicated" doesn\'t even begin to describe their relationship when his ex-girlfriend transfers to their school, determined to take Nash back.\nSee, Sabine isn\'t just an ordinary girl. She\'s a mara, the living personification of a nightmare. She can read people\'s fears--and craft them into nightmares while her victims sleep. Feeding from human fear is how she survives.\nAnd Sabine isn\'t above scaring Kaylee and the entire school to death to get whatever--and whoever--she wants.', image_url='https://s.gr-assets.com/assets/nophoto/book/111x148-bcc042a9c91a29c1d680899eff700a03.png', isbn='', isbn13='9781921685781', num_pages='343', publication_year='2011', title='My Soul to Steal (Soul Screamers, #4)', average_rating='4.13', ratings_count='5', book_id2=None, sentiment_count=None, sentime

In [125]:
# if rating_avg is null, set it to 0.5
dfBooks = dfBooks.withColumn("sentiment_avg", when(dfBooks.sentiment_avg.isNull(), 0.5).otherwise(dfBooks.sentiment_avg))
dfBooks = dfBooks.withColumn("sentiment_count", when(dfBooks.sentiment_count.isNull(), 0).otherwise(dfBooks.sentiment_count))
dfBooks = dfBooks.withColumn("sentiment_sum", when(dfBooks.sentiment_sum.isNull(), 0).otherwise(dfBooks.sentiment_sum))
# remove book_id2
dfBooks = dfBooks.drop("book_id2")
dfBooks.head(5)

[Row(authors=['415967'], book_id='10695227', description='Trying to work things out with Nash--her maybe boyfriend--is hard enough for Kaylee Cavanaugh. She can\'t just pretend nothing happened. But "complicated" doesn\'t even begin to describe their relationship when his ex-girlfriend transfers to their school, determined to take Nash back.\nSee, Sabine isn\'t just an ordinary girl. She\'s a mara, the living personification of a nightmare. She can read people\'s fears--and craft them into nightmares while her victims sleep. Feeding from human fear is how she survives.\nAnd Sabine isn\'t above scaring Kaylee and the entire school to death to get whatever--and whoever--she wants.', image_url='https://s.gr-assets.com/assets/nophoto/book/111x148-bcc042a9c91a29c1d680899eff700a03.png', isbn='', isbn13='9781921685781', num_pages='343', publication_year='2011', title='My Soul to Steal (Soul Screamers, #4)', average_rating='4.13', ratings_count='5', sentiment_count=0, sentiment_sum=0, sentimen

In [126]:
dfBookIds = spark.read.csv(bookidsPath, header=True, inferSchema=True)
dfBookIds.head(5)
# rename to book_id2
dfBookIds = dfBookIds.withColumnRenamed("book_id", "book_id2")

In [127]:
# keep from dfBooks only bookids in dfBookIds
#print(dfBooks.count())
dfBooks = dfBooks.join(dfBookIds, dfBooks.book_id == dfBookIds.book_id2, "inner")
#dfBooks.head(5)
#print(dfBooks.count())

In [128]:
#print(dfBooks.count())

In [129]:
#dfBooks.head(5)

In [130]:
# remove bookid2 column
dfBooks = dfBooks.drop("book_id2")
#dfBooks.head(5)

In [131]:
dfGenres = spark.read.json(genresPath)
dfGenres.head(5)

[Row(book_id='5333265', genres=Row(children=None, comics, graphic=None, fantasy, paranormal=None, fiction=None, history, historical fiction, biography=1, mystery, thriller, crime=None, non-fiction=None, poetry=None, romance=None, young-adult=None)),
 Row(book_id='1333909', genres=Row(children=None, comics, graphic=None, fantasy, paranormal=None, fiction=219, history, historical fiction, biography=5, mystery, thriller, crime=None, non-fiction=None, poetry=None, romance=None, young-adult=None)),
 Row(book_id='7327624', genres=Row(children=None, comics, graphic=None, fantasy, paranormal=31, fiction=8, history, historical fiction, biography=None, mystery, thriller, crime=1, non-fiction=None, poetry=1, romance=None, young-adult=None)),
 Row(book_id='6066819', genres=Row(children=None, comics, graphic=None, fantasy, paranormal=None, fiction=555, history, historical fiction, biography=None, mystery, thriller, crime=10, non-fiction=None, poetry=None, romance=23, young-adult=None)),
 Row(book_i

In [132]:
# rename to book_id2
dfGenres = dfGenres.withColumnRenamed("book_id", "book_id2")


In [133]:
def convertGenres(genres):
    genreList = []
    for genre in genres.__fields__:
        if genres[genre] is not None:
            if isinstance(genres[genre], bool):
                genreList.append(genre)
            elif isinstance(genres[genre], int):
                genreList.append(str(genre))
            else:
                genreList.append(str(genres[genre]))
    return ", ".join(genreList)

convertGenresUDF = udf(convertGenres, StringType())

dfGenres = dfGenres.withColumn("genres", convertGenresUDF(dfGenres.genres))


In [134]:
dfGenres.head(5)

[Row(book_id2='5333265', genres='history, historical fiction, biography'),
 Row(book_id2='1333909', genres='fiction, history, historical fiction, biography'),
 Row(book_id2='7327624', genres='fantasy, paranormal, fiction, mystery, thriller, crime, poetry'),
 Row(book_id2='6066819', genres='fiction, mystery, thriller, crime, romance'),
 Row(book_id2='287140', genres='non-fiction')]

In [135]:
# join genres with books
dfBooks = dfBooks.join(dfGenres, dfBooks.book_id == dfGenres.book_id2, "inner")
# remove book_id2
dfBooks = dfBooks.drop("book_id2")
#dfBooks.head(5)

In [136]:
# rename image_url to imageUrl , publication_year to year, book_id to bookId, rating_avg to avgRating, num_pages to numPages, ratings_count to ratingCount
dfBooks = dfBooks.withColumnRenamed("image_url", "imageUrl").withColumnRenamed("publication_year", "year").withColumnRenamed("book_id", "bookId").withColumnRenamed("average_rating", "avgRating").withColumnRenamed("num_pages", "numPages").withColumnRenamed("ratings_count", "ratingCount").withColumnRenamed("rating_sum", "ratingSum").withColumnRenamed("rating_count", "ratingCount").withColumnRenamed("sentiment_avg", "sentimentAvg").withColumnRenamed("sentiment_count", "sentimentCount").withColumnRenamed("sentiment_sum", "sentimentSum")


In [137]:
# convert year to int, avgRating to float, numPages to int, ratingCount to int
dfBooks = dfBooks.withColumn("year", dfBooks.year.cast(IntegerType())).withColumn("avgRating", dfBooks.avgRating.cast(FloatType())).withColumn("numPages", dfBooks.numPages.cast(IntegerType())).withColumn("ratingCount", dfBooks.ratingCount.cast(IntegerType()))

In [138]:
# check if any ratingSum is greater than 1
# dfBooks.filter(dfBooks.avgRating > 1).count()

1216881

In [139]:
#dfBooks.head(5)

In [140]:
# save as 20 json file
#dfBooks.coalesce(1).write.format('json').save('BooksUp.json', mode='overwrite')
dfBooks.coalesce(20).write.format('json').save('BooksUp.json', mode='overwrite')

In [142]:
# "11870085"
# "12578077"
# "13335037"
# "15507958"
# "345627"
# "11235712"
# "13450339"
# "1582996"
# "15881"
# "17167166"
# "17927395"
booksIdsRatingsEx = ["11870085", "12578077", "13335037", "15507958", "345627", "11235712", "13450339", "1582996", "15881", "17167166", "17927395"]
# get the ratings for the books in booksIdsRatingsEx
dfBooks.filter(dfBooks.bookId.isin(booksIdsRatingsEx)).select("bookId", "avgRating", "ratingCount").show()


+--------+---------+-----------+
|  bookId|avgRating|ratingCount|
+--------+---------+-----------+
|11235712|     4.15|     441530|
|11870085|     4.26|    2429317|
|12578077|     4.23|     168186|
|13335037|     4.23|    1962813|
|13450339|     4.14|      25769|
|15507958|     4.27|     609327|
| 1582996|     4.21|     550237|
|   15881|     4.38|    1821802|
|17167166|     4.49|     169307|
|17927395|     4.71|     120403|
|  345627|     4.14|     253685|
+--------+---------+-----------+



----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 50719)
Traceback (most recent call last):
  File "c:\Users\ahmed\AppData\Local\Programs\Python\Python39\lib\socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "c:\Users\ahmed\AppData\Local\Programs\Python\Python39\lib\socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "c:\Users\ahmed\AppData\Local\Programs\Python\Python39\lib\socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "c:\Users\ahmed\AppData\Local\Programs\Python\Python39\lib\socketserver.py", line 720, in __init__
    self.handle()
  File "c:\Users\ahmed\AppData\Local\Programs\Python\Python39\lib\site-packages\pyspark\accumulators.py", line 281, in handle
    poll(accum_updates)
  File "c:\Users\ahmed\AppData\Local\Programs\Python\Python39\lib\site-